In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import cv2
import numpy as np
from tensorflow.keras.layers import Dropout

In [2]:
bleeding_folder = r"C:\Users\SHIVANSH SHARMA\Desktop\projects\hack\bleeding"  # Replace with the path to your bleeding dataset
non_bleeding_folder = r"C:\Users\SHIVANSH SHARMA\Desktop\projects\non_bleeding"  # Replace with the path to your non-bleeding dataset

bleeding_images = [cv2.imread(os.path.join(bleeding_folder, img)) for img in os.listdir(bleeding_folder)]
non_bleeding_images = [cv2.imread(os.path.join(non_bleeding_folder, img)) for img in os.listdir(non_bleeding_folder)]

# Create labels (1 for bleeding, 0 for non-bleeding)
bleeding_labels = np.ones(len(bleeding_images))
non_bleeding_labels = np.zeros(len(non_bleeding_images))

# Combine the datasets
images = np.concatenate([bleeding_images, non_bleeding_images], axis=0)
labels = np.concatenate([bleeding_labels, non_bleeding_labels], axis=0)

In [3]:
# Create empty lists to store images and labels
images = []
labels = []

# Load and preprocess bleeding images
for filename in os.listdir(bleeding_folder):
    if filename.endswith('.png'):
        image = cv2.imread(os.path.join(bleeding_folder, filename))
        image = cv2.resize(image, (224, 224))  # Resize images to a consistent size (e.g., 224x224)
        image = image / 255.0  # Normalize pixel values to the range [0, 1]
        images.append(image)
        labels.append(1)  # Assign a label of 1 for bleeding images

# Load and preprocess non-bleeding images
for filename in os.listdir(non_bleeding_folder):
    if filename.endswith('.png'):
        image = cv2.imread(os.path.join(non_bleeding_folder, filename))
        image = cv2.resize(image, (224, 224))  # Resize images to a consistent size (e.g., 224x224)
        image = image / 255.0  # Normalize pixel values to the range [0, 1]
        images.append(image)
        labels.append(0)  # Assign a label of 0 for non-bleeding images

# Convert lists to NumPy arrays
X = np.array(images)
y = np.array(labels)


In [4]:
print("Number of loaded images:", len(images))

print("Number of labels:", len(labels))


Number of loaded images: 2618
Number of labels: 2618


In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Assuming X_train contains your training image data
sample_image = X_train[0]  # Take the first image as a sample
image_height, image_width, _ = sample_image.shape


In [7]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification, so use sigmoid activation
])

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
epochs = 10  # Set the number of training epochs (you can adjust this number as needed)

# Now you can use the 'epochs' variable in the 'model.fit' function
model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))



Epoch 1/10
66/66 [==============================] - 121s 2s/step - loss: 0.5314 - accuracy: 0.7880 - val_loss: 0.1769 - val_accuracy: 0.9313
Epoch 2/10
66/66 [==============================] - 114s 2s/step - loss: 0.1437 - accuracy: 0.9479 - val_loss: 0.1187 - val_accuracy: 0.9637
Epoch 3/10
66/66 [==============================] - 115s 2s/step - loss: 0.0775 - accuracy: 0.9718 - val_loss: 0.1360 - val_accuracy: 0.9389
Epoch 4/10
66/66 [==============================] - 113s 2s/step - loss: 0.0472 - accuracy: 0.9828 - val_loss: 0.1864 - val_accuracy: 0.9466
Epoch 5/10
66/66 [==============================] - 114s 2s/step - loss: 0.0382 - accuracy: 0.9871 - val_loss: 0.1104 - val_accuracy: 0.9637
Epoch 6/10
66/66 [==============================] - 103s 2s/step - loss: 0.0132 - accuracy: 0.9967 - val_loss: 0.1354 - val_accuracy: 0.9523
Epoch 7/10
66/66 [==============================] - 110s 2s/step - loss: 0.0156 - accuracy: 0.9952 - val_loss: 0.0702 - val_accuracy: 0.9809
Epoch 8/10
66

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Assuming 'y_val' contains true labels and 'y_pred' contains predicted labels
y_pred = model.predict(X_val)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate metrics
accuracy = accuracy_score(y_val, y_pred_binary)
precision = precision_score(y_val, y_pred_binary)
recall = recall_score(y_val, y_pred_binary)
f1 = f1_score(y_val, y_pred_binary)
roc_auc = roc_auc_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred_binary)

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print("Confusion Matrix:")
print(conf_matrix)


17/17 [==============================] - 7s 379ms/step
Accuracy: 0.9771
Precision: 0.9846
Recall: 0.9696
F1-Score: 0.9770
ROC-AUC: 0.9944
Confusion Matrix:
[[257   4]
 [  8 255]]
